In [1]:
import pandas as pd

from config import client_id, client_secret

from tensorflow import keras 
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from pathlib import Path
import sklearn as skl
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
file_path = Path("./Test-Data/final_genre_12k.csv")
df1 = pd.read_csv(file_path)

In [5]:
client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [19]:
track_name = input("Enter a track name: ")

In [31]:
# Search for the track
results = sp.search(q='edemame', type="track", limit=1)

In [32]:
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=edemame&type=track&offset=0&limit=1',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/41X1TR6hrK8Q2ZCpp2EqCz'},
       'href': 'https://api.spotify.com/v1/artists/41X1TR6hrK8Q2ZCpp2EqCz',
       'id': '41X1TR6hrK8Q2ZCpp2EqCz',
       'name': 'bbno$',
       'type': 'artist',
       'uri': 'spotify:artist:41X1TR6hrK8Q2ZCpp2EqCz'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/2IDLDx25HU1nQMKde4n61a'},
       'href': 'https://api.spotify.com/v1/artists/2IDLDx25HU1nQMKde4n61a',
       'id': '2IDLDx25HU1nQMKde4n61a',
       'name': 'Rich Brian',
       'type': 'artist',
       'uri': 'spotify:artist:2IDLDx25HU1nQMKde4n61a'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      

In [20]:
# Search for the track
results = sp.search(q=track_name, type="track", limit=1)

# Get the first track from the search results
track = results["tracks"]["items"][0]

# Get the track's audio features
features = sp.audio_features(track["id"])[0]

# Create a Pandas DataFrame with one row
df2 = pd.DataFrame([features])



In [21]:
df2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.815,0.848,4,-4.704,0,0.115,0.023,0.000162,0.0265,0.684,106.032,audio_features,1uMHCAyGmHqyygoNRuo7MV,spotify:track:1uMHCAyGmHqyygoNRuo7MV,https://api.spotify.com/v1/tracks/1uMHCAyGmHqy...,https://api.spotify.com/v1/audio-analysis/1uMH...,133707,4


In [9]:
X = df1.drop("primary_genre", axis=1)
X.head()
y = df1["primary_genre"].values

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=85)

In [11]:


# Preprocess the data (note: you'll need to define the X_scaler variable in your script as well)
model_X = df2.drop(columns={'type','id','uri','track_href','analysis_url'})

In [12]:

X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(model_X)
X_scaled = X_scaled.reshape(-1, 13)

In [13]:
model = keras.models.load_model('test_model.h5')

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-01-05 13:23:46.106802: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-05 13:23:46.106823: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:
prediction = model.predict(X_scaled)

2023-01-05 13:23:46.363421: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-05 13:23:46.404495: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 238ms/step


In [15]:
prediction_index = np.argmax(prediction, axis=1)

In [16]:
y_original = y_train

In [17]:
prediction_decoded = [y_original[prediction_index[i]] for i in range(prediction_index.shape[0])]

In [30]:
prediction_string = "".join(prediction_decoded)
prediction_string = prediction_string.replace('[','').replace(']','')

In [29]:
prediction_string

'Pop'